In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

from openpyxl import Workbook

In [3]:
driver = webdriver.Chrome(service= Service(ChromeDriverManager().install()))
url = "https://www.naver.com/"
driver.get(url)

In [4]:
def returnText(selector):
    try:
        text = driver.find_element(By.CSS_SELECTOR, selector).text
    except:
        text = None
    return(text)

In [5]:
searchBox = driver.find_element(By.XPATH,'//*[@id="query"]')
searchBox.send_keys("식당")
time.sleep(1)
searchBox.send_keys(Keys.RETURN)
map = driver.find_element(By.XPATH,'//*[@id="lnb"]/div[1]/div/div[1]/div/div[1]/div[1]/a').click()
time.sleep(1)

In [6]:
driver.switch_to.window(driver.window_handles[-1])
time.sleep(1)
iFrame = driver.find_element(By.ID,'searchIframe')
driver.switch_to.frame(iFrame)
time.sleep(0.3)


In [8]:
restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')

last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
while True:
    driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
    new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
    if new_height == last_height:
        break
    last_height = new_height
    time.sleep(0.1)

In [20]:
restaurants = []
i = 1
while i<10:
    driver.switch_to.default_content()
    iFrame = driver.find_element(By.ID,'searchIframe')
    driver.switch_to.frame(iFrame)
    currentPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
    try: 
        restaurant = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]/ul/li[%d]/div[1]/a/div/div/span[1]'%i)
        restaurant.click()
        time.sleep(1)
        driver.switch_to.default_content()
        iFrame = driver.find_element(By.ID,'entryIframe')
        driver.switch_to.frame(iFrame)
        restaurantName = returnText('#_title > div > span.GHAhO')
        restaurantTags = returnText('#_title > div > span.lnJFt')
        restaurantRating = returnText('#app-root > div > div > div > div.place_section.no_margin.OP4V8 > div.zD5Nm.undefined > div.dAsGb > span.PXMot.LXIwF')
        time.sleep(0.5)
        restaurantAddress = returnText('#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.tQY7D > div > a > span.LDgIH')
        restaurantCallNumber = returnText('#app-root > div > div > div > div:nth-child(5) > div > div:nth-child(2) > div.place_section_content > div > div.O8qbU.nbXkr > div > span.xlx7Q')
        restaurants.append({
            'name': restaurantName,
            'tags': restaurantTags,
            'rating': restaurantRating,
            'address': restaurantAddress,
            'callNumber': restaurantCallNumber
        })
        i+=1
    except:
        nextButton = driver.find_element(By.XPATH,'//*[@id="app-root"]/div/div[2]/div[2]/a[6]')
        nextButton.click()
        time.sleep(0.7)
        nextPage = driver.find_element(By.CLASS_NAME,"mBN2s.qxokY").text
        if nextPage == currentPage: break
        restaurantGroup = driver.find_element(By.XPATH,'//*[@id="_pcmap_list_scroll_container"]')
        last_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
        while True:
            driver.execute_script("arguments[0].scrollBy(0, 1000)", restaurantGroup)
            new_height = driver.execute_script("return arguments[0].scrollTop;", restaurantGroup)
            if new_height == last_height:
                break
            last_height = new_height
            time.sleep(0.2)
        i = 1
print(restaurants)
print(len(restaurants))

[{'name': '노브랜드버거 성남태평점', 'tags': '햄버거', 'rating': '별점\n4.34', 'address': '경기 성남시 수정구 제일로 165 1층', 'callNumber': '031-751-3690'}, {'name': '스시마루', 'tags': '초밥,롤', 'rating': '별점\n4.4', 'address': '경기 성남시 수정구 탄리로 92 1층스시마루', 'callNumber': '031-757-8630'}, {'name': '스시마루', 'tags': '초밥,롤', 'rating': '별점\n4.4', 'address': '경기 성남시 수정구 탄리로 92 1층스시마루', 'callNumber': '031-757-8630'}, {'name': '히노아지 성남신흥점', 'tags': '일본식라면', 'rating': None, 'address': '경기 성남시 수정구 산성대로 267 신흥역시네마타워 7층 703-1호', 'callNumber': '031-747-5349'}, {'name': '가마솥순대국밥 성남신흥점', 'tags': '순대,순댓국', 'rating': None, 'address': '경기 성남시 수정구 시민로 144 1층 가마솥순대국밥 성남 신흥점', 'callNumber': '0507-1390-2641'}, {'name': '하루엔소쿠 신흥롯데시네마점', 'tags': '돈가스', 'rating': '별점\n4.27', 'address': '경기 성남시 수정구 산성대로 267 2층 221, 222호 돈까스 맛집 하루엔소쿠', 'callNumber': '031-744-2585'}, {'name': '춘리마라탕 신흥역점', 'tags': '마라탕', 'rating': None, 'address': '경기 성남시 수정구 산성대로 271-1 2층', 'callNumber': '0507-1409-8886'}, {'name': '참미루축산물정육식당', 'tags': '정육식당', 'rating': '별점\n4.3

In [22]:
wb = Workbook()
ws = wb.active
ws.title = "Restaurants"
column = ['name','tags','rating','adress','callNumber']
ws.append(column)
for i in range(len(restaurants)):
    row = [restaurants[i]['name'],restaurants[i]['tags'],restaurants[i]['rating'],restaurants[i]['address'],restaurants[i]['callNumber']]
    ws.append(row)
wb.save('restaurant list.xlsx')
wb.close()